In [1]:
import requests
import urllib.parse
from bs4 import BeautifulSoup
from time import sleep
import copy
import pandas as pd

In [2]:
response = requests.get('https://www.bilibili.tv/trending/anime') 

 
print(response)
print(response.status_code, response.reason)
print(response.headers)

<Response [200]>
200 OK
{'Content-Type': 'text/html', 'Content-Encoding': 'gzip', 'IDC': 'sgp001', 'Date': 'Fri, 27 Oct 2023 09:10:30 GMT', 'Content-Length': '14349', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Set-Cookie': 'buvid3=a4ad9cbe-43b3-4001-a68a-60c4bb2e000762433infoc; path=/; expires=Sat, 26 Oct 2024 09:10:30 GMT; domain=.bilibili.tv, bstar-web-lang=th; path=/; expires=Sat, 28 Oct 2023 09:10:30 GMT; domain=.bilibili.tv', 'X-Cache-Webcdn': 'aka', 'Cache-Control': 'no-cache, no-transform'}


In [3]:
response.encoding = 'utf-8'

In [4]:
html = response.text
print(html[0:500])

<!DOCTYPE html>
<html lang="th" style="background: #fafafa" data-head-attrs="lang,style" lang="en">
  <head>
    <script type="module" crossorigin src="//p.bstarstatic.com/fe-static/bstar-web-new/client/assets/polyfills-20dfc4b6.js"></script>

    <script>window.__BSTAR__METADATA__={version:"1.46.0"}</script>

    <meta charset="UTF-8" />
    <meta name="spm_prefix" content="0">
    <meta name="referrer" content="no-referrer-when-downgrade">
    <meta name="viewport" content="width=device-width,


In [5]:
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify()[0:500])

<!DOCTYPE html>
<html data-head-attrs="lang,style" lang="en" style="background: #fafafa">
 <head>
  <script crossorigin="" src="//p.bstarstatic.com/fe-static/bstar-web-new/client/assets/polyfills-20dfc4b6.js" type="module">
  </script>
  <script>
   window.__BSTAR__METADATA__={version:"1.46.0"}
  </script>
  <meta charset="utf-8"/>
  <meta content="0" name="spm_prefix"/>
  <meta content="no-referrer-when-downgrade" name="referrer"/>
  <meta content="width=device-width, user-scalable=no, initial-


In [6]:
target_a = soup.find("span",class_ = "trending-card__title-text trending-card__title-text--anime")

In [7]:
print(target_a)

<span class="trending-card__title-text trending-card__title-text--anime" data-v-7926129b="">ประเภทโดยรวม</span>


In [8]:
print(target_a.prettify()[0:200])

<span class="trending-card__title-text trending-card__title-text--anime" data-v-7926129b="">
 ประเภทโดยรวม
</span>



In [9]:
target_s = soup.find("ul",class_="trending-videolist")

In [10]:
print(target_s.prettify()[0:2000])

<ul class="trending-videolist" data-v-4f5128ec="">
 <!--[-->
 <li data-v-4f5128ec="">
  <div class="bstar-video-card bstar-video-card--column bstar-video-card--ogv" data-v-4f5128ec="" style="grid-template-columns:240px 1fr;">
   <div class="bstar-video-card__cover-wrap">
    <div class="bstar-video-card__cover bstar-video-card__cover--normal" style="width:240px;height:135px;">
     <a href="//www.bilibili.tv/th/play/37976" target="_blank">
      <picture class="bstar-image bstar-video-card__cover-img">
       <!-- -->
       <source srcset="https://pic.bstarstatic.com/ogv/8fb92de3e88b08d3c4283d1b08fac25df5cadea0.png@240w_135h_1e_1c_90q.webp" type="image/webp"/>
       <img alt="วันพีซ" class="bstar-image__img" loading="lazy" src="https://pic.bstarstatic.com/ogv/8fb92de3e88b08d3c4283d1b08fac25df5cadea0.png@240w_135h_1e_1c_90q"/>
      </picture>
     </a>
     <!--[-->
     <div class="bstar-video-card__cover-mask">
      <span class="bstar-video-card__cover-mask-text bstar-video-card__

In [11]:
target_d = target_s.find("p",class_="bstar-video-card__title bstar-video-card__title--small bstar-video-card__title--bold")

In [12]:
target_d.text

'วันพีซ'

In [13]:
movies = []

In [14]:
for li in target_s('li'):
    series = {}
    series["name"] = li.img["alt"]
    series["views"] = li.find("span",class_="videocard-slot__playinfo-text videocard-slot__playinfo-vv").get_text()
    series["comment"] = li.find("span",class_="videocard-slot__playinfo-text videocard-slot__playinfo-dm").get_text()
    series["recent"] = li.find("span",class_="bstar-video-card__cover-mask-text bstar-video-card__cover-mask-text--bold").get_text()
    movies.append(series)

In [16]:
table_movies = pd.DataFrame(movies)

In [17]:
table_movies

,name,views,comment,recent
0,วันพีซ,2.1B,10.7M,อัปเดตเป็นตอนที่ 1080
1,มหาเวทย์ผนึกมาร ซีซัน 2,47.6M,83.9K,อัปเดตเป็นตอนที่ 14
2,ชีวิตไม่ต้องเด่น ขอแค่เป็นเทพในเงา ภาคที่ 2,5.1M,19.7K,อัปเดตเป็นตอนที่ 4
3,SPY x FAMILY ซีซั่น 2,2.6M,12.9K,อัปเดตเป็นตอนที่ 28
4,จอมมารเกิดใหม่ วิทยาลัยผู้พิทักษ์,2.9M,10.7K,อัปเดตเป็นตอนที่ 4
5,มหาวิบัตินครดับสูญ,1.2M,5.7K,อัปเดตเป็นตอนที่ 3
6,ตำนานนักล่ามังกร,2.3M,10.8K,อัปเดตเป็นตอนที่ 4
7,ผู้กล้าโล่ผงาด ซีซัน 3,1.3M,6.0K,อัปเดตเป็นตอนที่ 3
8,นารูโตะ ตำนานวายุสลาตัน,760.7M,2.9M,เสร็จแล้ว
9,มหาเวทย์ผนึกมาร,161.2M,440.1K,เสร็จแล้ว
